In [121]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import RobustScaler,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pickle as pkl
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,root_mean_squared_error

In [106]:
df=pd.read_csv(r'car_sales_data.csv')

In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Manufacturer         50000 non-null  object 
 1   Model                50000 non-null  object 
 2   Engine size          50000 non-null  float64
 3   Fuel type            50000 non-null  object 
 4   Year of manufacture  50000 non-null  int64  
 5   Mileage              50000 non-null  int64  
 6   Price                50000 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 2.7+ MB


In [108]:
df.head()

,Manufacturer,Model,Engine size,Fuel type,Year of manufacture,Mileage,Price
0,Ford,Fiesta,1.0,Petrol,2002,127300,3074
1,Porsche,718 Cayman,4.0,Petrol,2016,57850,49704
2,Ford,Mondeo,1.6,Diesel,2014,39190,24072
3,Toyota,RAV4,1.8,Hybrid,1988,210814,1705
4,VW,Polo,1.0,Petrol,2006,127869,4101


In [109]:
cat_col=df.select_dtypes(exclude='number').columns
df[cat_col]=df[cat_col].astype('category')

In [110]:
x=df.drop(columns='Price')
y=df['Price']

In [111]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [112]:
num=x_train.select_dtypes(include='number').columns
cat=x_train.select_dtypes(exclude='number').columns

In [113]:
num_pipleline=Pipeline(
    steps=[
        ('num_scaling',RobustScaler())
    ]
)


cat_pipleline=Pipeline(
    steps=[
        ('car_encoding',OneHotEncoder(handle_unknown='ignore'))
    ]
)

In [114]:
preprocessing=ColumnTransformer(
    transformers=[
        ('num_pipleline',num_pipleline,num),
        ('cat_pipleline',cat_pipleline,cat)
    ]
)

In [122]:
main_pipeline=Pipeline(
    steps=[
        ("preprocessing",preprocessing),
        ('model',XGBRegressor())
    ]
)

In [123]:
main_pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_pipleline',
                                                  Pipeline(steps=[('num_scaling',
                                                                   RobustScaler())]),
                                                  Index(['Engine size', 'Year of manufacture', 'Mileage'], dtype='object')),
                                                 ('cat_pipleline',
                                                  Pipeline(steps=[('car_encoding',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Manufacturer', 'Model', 'Fuel type'], dtype='object'))])...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [124]:
main_pipeline.score(x_train,y_train)

0.9993675947189331

In [125]:
main_pipeline.score(x_test,y_test)

0.998959481716156

In [126]:
y_pred = main_pipeline.predict(x_test)

print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", root_mean_squared_error(y_test, y_pred))

MAE: 322.7107238769531
RMSE: 531.381591796875


In [127]:
with open('model.pkl', 'wb') as file:
    pkl.dump(main_pipeline, file)